In [46]:
from cvxopt import matrix
from cvxopt import solvers
def SVM_primal(input,c):
    # Load input
    row_length,col_length = input.shape
    row_training  = row_length

    # Initialize training
    train_x = input[:,0:col_length-1]
    train_x = np.concatenate((train_x,np.ones((row_training,1))),axis = 1)#Add ones column as bias
    train_y = input[:,col_length-1:col_length]
    feature_num = train_x.shape[1] #11

    P = np.eye(feature_num-1)
    P = np.concatenate((P,np.zeros((feature_num-1,row_training+1))),axis = 1)
    P = np.concatenate((P,np.zeros((row_training+1,row_training+feature_num))),axis = 0) #347 x 347

    q = c * np.concatenate((np.zeros((1,feature_num)),train_y.T),axis=1)

    h = np.concatenate((-np.ones((row_training,1)),np.zeros((row_training,1))),axis=0)

    x = np.concatenate((train_x,np.zeros((row_training,feature_num))),axis=0) #336 x 347
    eye = np.eye(row_training) # 336 x 336
    eye = np.vstack((eye,eye)) #Add ones column as bias -> already added above
    x = np.concatenate((x,eye),axis=1) #672 x 347
    y = np.concatenate((train_y,train_y),axis=0)
    G = -y * x

    # Quadprop for Primal

    # Define QP parameters (directly)
    P_ = matrix(P)
    q_ = matrix(q.T)
    G_ = matrix(G)
    h_ = matrix(h)

    # Construct the QP, invoke solver
    sol = solvers.qp(P_,q_,G_,h_)

    # Get weight and b
    train_w = np.array(sol['x'])
    train_w = train_w[0:feature_num]
#     f = np.dot(train_x,train_w)
#     f[f < 0] = -1
#     f[f > 0] =  1 

#     #print training accurary
#     print ("Accuracy: "+str(100*np.count_nonzero(train_y*f+1)/row_training))
    return train_w

# def load_data(filename):
#     input = np.loadtxt(filename, delimiter=',')
#     row_length,col_length = input.shape

#     # Initialize training
#     train_x = input[:,0:col_length-1]
#     train_x = np.concatenate((train_x,np.ones((row_length,1))),axis = 1)#Add ones column as bias
#     train_y = input[:,col_length-1:col_length]
#     return [train_x,train_y]

def testing(input,w):
    row_length,col_length = input.shape
    train_x = input[:,0:col_length-1]
    train_x = np.concatenate((train_x,np.ones((row_length,1))),axis = 1)#Add ones column as bias
    train_y = input[:,col_length-1:col_length]

    f = np.dot(train_x,w)
    f[f < 0] = -1
    f[f > 0] =  1 
    accuracy = 100*np.count_nonzero(train_y*f+1)/row_length
    return accuracy

In [19]:
import numpy as np
# Read data
filename = "spam_train.data"
data = np.loadtxt(filename,delimiter=',')
# Define input and output
num_data = data.shape[0]
num_feature = data.shape[1]-1
X = data[:,0:num_feature]
Y = data[:,num_feature]
# Construct matrix W - sample covariance matrix 
X_mean = np.mean(X,axis=0)
X_mean = np.reshape(X_mean,(57,1))
W = X - np.dot(np.ones((num_data,1)),X_mean.T)
# Find SVD of W - covariance matrix
U, s, V = np.linalg.svd(W, full_matrices=True)
# eigen vectors and eigen value of covariance matrix 
eigen_vec = U
# s = np.reshape(s,(s.shape[0],1))
# eigen_value = np.dot(s,s.T)
eigen_value = s * s
# Top 6 eigen value
print("Top 6 eigen values: ",eigen_value[:6])

Top 6 eigen values:  [  1.52029434e+09   1.28884476e+08   3.44289156e+06   1.68285514e+04
   1.16228317e+04   8.97974605e+03]


In [68]:
# PCA and SVM
# six eigen vector
V_r = V[:,0:5]
X_r = np.dot(X,V_r)
Y = np.reshape(Y,(Y.shape[0],1))
Y[Y==0] = -1
SVM_data = np.concatenate((X_r,Y),axis=1) #672 x 347
c= 1000
SVM_w = SVM_primal(SVM_data,c)

     pcost       dcost       gap    pres   dres
 0: -1.8705e+09  7.1016e+09  5e+10  5e+03  2e+02
 1:  4.9790e+09 -4.5201e+09  1e+10  8e+02  3e+01
 2:  1.9471e+09 -3.1577e+07  2e+09  9e+00  4e-01
 3:  3.0381e+07  1.0260e+06  3e+07  1e-01  6e-03
 4:  2.9934e+06  1.0856e+06  2e+06  9e-03  4e-04
 5:  2.4098e+06  1.4174e+06  1e+06  4e-03  2e-04
 6:  2.1215e+06  1.5756e+06  5e+05  2e-03  7e-05
 7:  1.9809e+06  1.6549e+06  3e+05  9e-04  4e-05
 8:  1.8849e+06  1.7108e+06  2e+05  4e-04  2e-05
 9:  1.8234e+06  1.7485e+06  8e+04  2e-04  7e-06
10:  1.7984e+06  1.7646e+06  3e+04  7e-05  3e-06
11:  1.7870e+06  1.7706e+06  2e+04  5e-16  1e-13
12:  1.7801e+06  1.7767e+06  3e+03  5e-16  1e-13
13:  1.7786e+06  1.7781e+06  5e+02  6e-16  7e-14
14:  1.7784e+06  1.7783e+06  1e+02  6e-16  1e-12
15:  1.7784e+06  1.7784e+06  2e+01  5e-16  5e-12
16:  1.7784e+06  1.7784e+06  2e+00  5e-16  6e-12
Optimal solution found.


In [67]:
testing(SVM_data,SVM_w) #c = 10

75.96666666666667

In [69]:
testing(SVM_data,SVM_w) #c = 1000

75.96666666666667